In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model

In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [15]:
image_name = 'bfimage'
image = ws.images[image_name]

### Onetime Provision AKS Cluster

In [7]:
prov_config = AksCompute.provisioning_configuration()

aks_name = 'bf-aks' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [8]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating.................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None
Wall time: 7min 26s


In [10]:
aks_target.agent_vm_size

'STANDARD_D3_V2'

#### SSL Config

In [12]:
# TODO

### Deploy webservice

In [13]:
aks_config = AksWebservice.deploy_configuration()

In [16]:
%%time
aks_service_name ='bf-aks-service'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running..............
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy
Wall time: 1min 21s


### Test webservice using run method

In [25]:
import pickle
import json

In [20]:
with open('../model/data/test_val.pkl', 'rb') as f:
    test_val = pickle.load(f)

In [23]:
with open('../model/azureml/model/outcome_var.pkl', 'rb') as f:
    target_col = pickle.load(f)

In [26]:
actual = test_val[target_col]
test_data = {k:v for k,v in test_val.items() if not k == target_col}
test_data_json = bytes(json.dumps(test_data), encoding='utf8')
test_data_json

b'{"User_ID": "1000001", "Product_ID": "P00248942", "Gender": "F", "Age": "0-17", "Occupation": "10", "City_Category": "A", "Stay_In_Current_City_Years": "2", "Marital_Status": "0", "Product_Category_1": "1", "Product_Category_2": "6.0", "Product_Category_3": "14.0"}'

In [28]:
aks_service.run(input_data=test_data_json)

17175

In [29]:
actual

15200

### Test webservice using http post request

In [31]:
key1, Key2 = aks_service.get_keys()
key1

'BxM0tMv4d9NATe4Yq4pgc6OgMtsSwqB5'

In [34]:
scoring_uri = aks_service.scoring_uri
scoring_uri

'http://52.191.187.136:80/api/v1/service/bf-aks-service/score'

In [36]:
import requests
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}
resp = requests.post(aks_service.scoring_uri, test_data_json, headers=headers)
resp.text

'17175'

### Cleanup

In [ ]:
aks_service.delete()